# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from API_Keys import gkey
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../Output_Data/cities.csv')
cities = cities.set_index("City ID")
cities

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
City ID,,,,,,,,,
0,Albany,42.6001,-73.9662,81.45,55,42,1.99,US,1661625729
1,Avarua,-21.2078,-159.7750,73.45,73,19,9.22,CK,1661625754
2,Orocué,4.7912,-71.3354,88.39,57,30,3.33,CO,1661626037
3,Husavik,66.0449,-17.3389,48.74,67,14,4.72,IS,1661626038
4,Bluff,-46.6000,168.3333,46.80,74,100,1.41,NZ,1661626038
...,...,...,...,...,...,...,...,...,...
589,Douglas,41.3000,-96.1670,85.87,56,0,12.66,US,1661626240
590,Soligalich,59.0778,42.2864,72.19,48,94,7.38,RU,1661626241
591,Jiuquan,39.7432,98.5174,64.81,51,100,7.54,CN,1661626241


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coordinates = cities[['Lat', 'Lng']]
humidity = cities['Humidity (%)']

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '1500px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

# Assign the heatmap layer to a variable
heatmap = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=humidity.max(), point_radius=3.5)

# Add the layer to the map
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Personal weather conditions: 75 <= Temp <= 80, Wind <= 10, Cloudiness <= 50, Humidity <= 70, not in the US
hotel_df = cities.loc[(cities['Max Temp (F)'] <= 80) & (cities['Max Temp (F)'] >= 75) & (cities['Wind Speed (mph)'] <= 10) & (cities['Cloudiness (%)'] <= 50) & (cities['Humidity (%)'] <= 70) & (cities['Country'] != 'US')]
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
City ID,,,,,,,,,
83,Caravelas,-17.7125,-39.2481,75.49,70,5,8.05,BR,1661626066
132,Nsanje,-16.9200,35.2620,76.51,36,4,4.74,MW,1661626084
145,Bat Khela,34.6167,71.9714,75.29,70,33,3.44,PK,1661625895
212,Cidreira,-30.1811,-50.2056,75.60,57,0,4.27,BR,1661626111
229,Zatoka,46.0725,30.4654,76.14,64,11,8.99,UA,1661626116
348,Sukhumi,43.0033,41.0153,75.24,68,0,3.91,GE,1661626155
374,Mahon,39.8885,4.2658,79.36,69,20,3.44,ES,1661626056
422,Yangi Marg`ilon,40.4272,71.7189,75.92,31,6,2.30,UZ,1661626181
451,Nchelenge,-9.3451,28.7340,76.05,42,31,4.38,ZM,1661626192


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ''

# Set up a parameters dictionary
params = {
    "keyword": 'hotel',
    "radius": 5000,
    "type": 'lodging',
    "key": gkey
}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # Identify coordinates for each city
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'

    # Base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        print(f'Appending hotel result for Index {index}: {row["City"]}')
        print(f'The closest hotel found was: {response["results"][0]["name"]}')
    except:
        hotel_df.loc[index, 'Hotel Name'] = 'N/A'
        print(f'No hotel found for Index {index}: {row["City"]}')

    print('--------------------------------')

C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_65696\4179773088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


Appending hotel result for Index 83: Caravelas
The closest hotel found was: Pousada Liberdade
--------------------------------
Appending hotel result for Index 132: Nsanje
The closest hotel found was: Shumba Lodge
--------------------------------
Appending hotel result for Index 145: Bat Khela
The closest hotel found was: Fort View hotel
--------------------------------
Appending hotel result for Index 212: Cidreira
The closest hotel found was: Pousada Lessa
--------------------------------
Appending hotel result for Index 229: Zatoka
The closest hotel found was: Bless Resort & Villa
--------------------------------
Appending hotel result for Index 348: Sukhumi
The closest hotel found was: Garuda hotel
--------------------------------
Appending hotel result for Index 374: Mahon
The closest hotel found was: HOTEL VICTORI
--------------------------------
Appending hotel result for Index 422: Yangi Marg`ilon
The closest hotel found was: TANTANA HOTEL FERGANA
------------------------------

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…